In [5]:
from src.training.training import train_model, train_and_save, load_model
from src.models.get_data import get_tensors

In [2]:
forecast_distribution = 'distr_trunc_normal'
distribution_1 = 'distr_trunc_normal'
distribution_2 = 'distr_log_normal'

loss = 'loss_CRPS_sample' # options: loss_CRPS_sample, loss_twCRPS_sample, loss_log_likelihood

chain_function = 'chain_function_normal_cdf_plus_constant' # options: chain_function_normal_cdf, chain_function_indicator, chain_function_normal_cdf_plus_constant
chain_function_mean = 13
chain_function_std = 2
chain_function_threshold = 15 # 12 / 15
chain_function_constant = 0.05

optimizer = 'Adam'
learning_rate = 0.03
folds = [1,2]
neighbourhood_size = 11
ignore = ['229', '285', '323']
epochs = 600

samples = 100
printing = True
pretrained = True
random_init = True

all_features = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot', 'spatial_variance']

location_features = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']

scale_features = ['spatial_variance']

In [3]:
models = []
for i in range(0, 10):
    model = train_model(
    forecast_distribution,
    loss,
    optimizer,
    learning_rate,
    folds,
    all_features,
    location_features,
    scale_features,
    neighbourhood_size,
    ignore,
    epochs,

    chain_function = chain_function,
    chain_function_mean = chain_function_mean,
    chain_function_std = chain_function_std,
    chain_function_constant = chain_function_constant,
    chain_function_threshold = chain_function_threshold,
    samples = samples,
    printing = printing,
    distribution_1 = distribution_1,
    distribution_2 = distribution_2,
    pretrained = pretrained,
    random_init = random_init
    )
    models.append(model)

Using random initialization for Truncated Normal distribution
Step: 0, Loss: 1.9398316144943237
Step: 1, Loss: 1.80514395236969
Step: 2, Loss: 1.674045443534851
Step: 3, Loss: 1.5507371425628662
Step: 4, Loss: 1.4390132427215576
Step: 5, Loss: 1.3362535238265991
Step: 6, Loss: 1.2457209825515747
Step: 7, Loss: 1.1655768156051636
Step: 8, Loss: 1.1001726388931274
Step: 9, Loss: 1.0455610752105713
Step: 10, Loss: 1.0037130117416382
Step: 11, Loss: 0.9730518460273743
Step: 12, Loss: 0.9504069089889526
Step: 13, Loss: 0.9380336999893188
Step: 14, Loss: 0.9306142926216125
Step: 15, Loss: 0.9336106777191162
Step: 16, Loss: 0.9356699585914612
Step: 17, Loss: 0.9428601861000061
Step: 18, Loss: 0.9517331123352051
Step: 19, Loss: 0.9588522911071777
Step: 20, Loss: 0.9639836549758911
Step: 21, Loss: 0.9702564477920532
Step: 22, Loss: 0.9737293124198914
Step: 23, Loss: 0.9720901846885681
Step: 24, Loss: 0.9697901606559753
Step: 25, Loss: 0.9659751653671265
Step: 26, Loss: 0.9599402546882629
Step: 

In [6]:

test_fold = 3
ignore = ['229', '285', '323']
X_test, y_test = get_tensors(neighbourhood_size, all_features, test_fold, ignore)
X_test = models[0].normalize_features(X_test)

In [9]:
for model in models:
    print(model.get_parameters())

{'a_tn': array([0.4003619], dtype=float32), 'b_tn': array([ 0.9648459 , -0.04112827, -0.14490141,  0.03871054,  0.02568408],
      dtype=float32), 'c_tn': array([1.3559366], dtype=float32), 'd_tn': array([1.0739466], dtype=float32)}
{'a_tn': array([0.41092145], dtype=float32), 'b_tn': array([ 0.9630575 , -0.03918617, -0.14253135,  0.03038079,  0.02198171],
      dtype=float32), 'c_tn': array([1.251305], dtype=float32), 'd_tn': array([1.1778133], dtype=float32)}
{'a_tn': array([0.40132084], dtype=float32), 'b_tn': array([ 0.965072  , -0.03776813, -0.1481072 ,  0.03171873,  0.02497572],
      dtype=float32), 'c_tn': array([1.3447578], dtype=float32), 'd_tn': array([1.0841472], dtype=float32)}
{'a_tn': array([1.0068308], dtype=float32), 'b_tn': array([ 0.8411719 , -0.35800368,  0.00148516, -0.14134721,  0.23461081],
      dtype=float32), 'c_tn': array([3.0507114], dtype=float32), 'd_tn': array([1.5716119], dtype=float32)}
{'a_tn': array([0.40105897], dtype=float32), 'b_tn': array([ 0.9643

In [10]:
for model in models:
    print("CRPS: ", model.CRPS(X_test, y_test, 10000))

CRPS:  tf.Tensor(0.8907267, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.8908204, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.8906335, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.9315458, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.89038336, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.89178014, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.8979401, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.8907616, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.8905352, shape=(), dtype=float32)
CRPS:  tf.Tensor(0.89081496, shape=(), dtype=float32)
